# pyspark dynamo db

In [2]:
import os
import ast
import logging
import pprint
import json
import time
import yaml
import datetime as dt
import pandas as pd
import numpy as np

import boto3
import pprint
import botocore.session
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.column import *
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr

pp = pprint.PrettyPrinter(indent = 1)
print("imported modules.")

with open("./../creds.json", "r") as g:
    creds = json.load(g)
    g.close()

with open("./../config.yaml", "r") as h:
    config = yaml.safe_load(h)
    h.close()

print("read config & creds files.")

s3 = boto3.client("s3")
athena = boto3.client("athena")
glue = boto3.client("glue")
my_lambda = boto3.client("lambda")
dynamo = boto3.client("dynamodb")
print("connected to boto3 clients.")

imported modules.
read config & creds files.
connected to boto3 clients.


## spark session

In [ ]:
try:
    spark = SparkSession.builder.appName("twitter") \
        .master("spark://{}:{}".format(creds["spark_host"], creds["spark_port"])) \
        .config("spark.executor.memory", "2048m") \
        .config("spark.executor.cores", "2") \
        .config("spark.streaming.concurrentJobs", "8") \
        .config("spark.local.dir", "/opt/workspace/tmp/driver/") \
        .config("spark.worker.dir", "/opt/workspace/tmp/executor/") \
        .config("spark.eventLog.enabled", "true") \
        .config("spark.eventLog.dir", "/opt/workspace/tmp/events/") \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .config("spark.jars.packages", config["extra_jar_list"]) \
        .config("spark.hadoop.fs.s3a.access.key", creds["aws_client"]) \
        .config("spark.hadoop.fs.s3a.secret.key", creds["aws_secret"]) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.hadoop.fs.s3a.buffer.dir", "/opt/workspace/tmp/blocks") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
        .enableHiveSupport() \
        .getOrCreate()

    sc = spark.sparkContext
    # index = 0
    sc.setLogLevel("WARN")
    # sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
    print("imported modules, created spark.")

except Exception as f:
    print("EXCEPTION: ")

## spark df

In [ ]:
try:
    df = spark.read.format("delta").option("header", True).load(filepath)
    # df.show()
    print("read df.")

except Exception as g:
    print("EXCEPTION: {}".format(g))

## pandas df

In [ ]:
df_pandas = df.toPandas()
df_pandas

In [ ]:
df_pandas.shape

## dynamodb

In [3]:
dynamo_client = boto3.client("dynamodb", region_name = "us-east-2")
dynamo_resource = boto3.resource("dynamodb", region_name = "us-east-2")
print("created dynamodb client & resource.")

created dynamodb client & resource.


In [5]:
tweets = dynamo_resource.Table("tweets")
response = tweets.scan()
count = response["Count"]
print("table {} has count {}".format(tweets.name, count))

table tweets has count 5552


In [7]:
# response["Items"]